In [15]:
import re
from datetime import datetime

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

First lets work out how many pages are on the tournament decklists page.

In [16]:
def get_number_of_pages():
    r = requests.get(f"https://fabtcg.com/decklists/?page=1")
    soup = BeautifulSoup(r.text, 'html.parser')
    last = int(soup.findAll("a", {"class": "page-link starling"})[-1]['href'].partition("=")[-1])
    return last

Now on each page we have to read the table, find all the links, and stitch them together.

In [17]:
def get_page(page):
    r = requests.get(f"https://fabtcg.com/decklists/?page={page}")
    soup = BeautifulSoup(r.text, 'html.parser')
    
    df = pd.read_html(f"https://fabtcg.com/decklists/?page={page}", flavor='bs4')[0]
    
    table = soup.find('table')
    links = {}
    for tr in table.findAll("tr"):
        trs = tr.findAll("td")
        for each in trs:
            try:
                a_text = re.sub('\s+',' ', each.find('a').contents[0])
                a_link = each.find('a')['href']
                links[a_text] = a_link
            except:
                pass

    def fetch_deck_url(row):
        try:
            deck = links[row['Decklist']]
        except:
            deck = None

        return deck

    def fetch_event_url(row):
        try:
            event = links[row['Event']]
        except:
            event = None

        return event
    
    df['deck_link'] = df.apply(fetch_deck_url, axis=1)
    df['event_link'] = df.apply(fetch_event_url, axis=1)
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df

Loop through the available pages and concat out the df.

In [18]:
def get_all_pages():
    dfs = []
    for i in range(1, get_number_of_pages()):
        dfs.append(get_page(i))
        
    return pd.concat(dfs).reset_index(drop=True)

In [19]:
df = get_all_pages()

In [20]:
df.to_csv("tournament_lists.csv")

Next we're going to use the cards we got from FABDB to get more data on card data.

In [21]:
card_df = pd.read_csv("cards.csv").drop(['image', 'resource', 'name'], axis=1)

FileNotFoundError: [Errno 2] File b'cards.csv' does not exist: b'cards.csv'

In [26]:
def enrich_deck(deck_df, card_df):
    return pd.merge(deck_df, card_df, left_on=['name_resource'], right_on=['name_resource'])

In [27]:
def get_deck(url):
    r = requests.get(url)
    soup = r.text
    frames = pd.read_html(url, flavor='bs4')
    
    metadata = {frames[0].iloc[:, 0].to_list()[i]: frames[0].iloc[:, 1].to_list()[i] for i in range(frames[0].shape[0])}
    equipment = frames[1].iloc[:, 0].str.extract("(?P<copies>.*?) x (?P<name>.*)")
    
    dfs = []
    for i in frames[2:]:
        dfs.append(i.iloc[:, 0].str.extract("(?P<copies>.*?) x (?P<name>.*) \((?P<resource>.*)\)"))    
    dfs.append(equipment)
    df = pd.concat(dfs)
    
    def transform_name(row):

        resource_map = {
            '3': '(Blue)',
            '2': '(Yellow)',
            '1': '(Red)',
            np.nan: '',
        }
        
        return f"{row['name']} {resource_map[row['resource']]}"
    
    df['name_resource'] = df.apply(transform_name, axis=1)
    
    df = enrich_deck(df, card_df)
    numeric_cols = ['copies', 'resource', 'attack', 'cost', 'defense', 'intellect', 'life']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    return metadata, df

In [32]:
releases = {
    'CRU': ('2020-08-28', datetime.now().strftime("%Y-%m-%d")),
    'ARC': ('2020-08-28', '2020-11-20'),
    'WTR': ('1970-01-01', '2020-11-20'),
    'ALL': ('1970-01-01', datetime.now().strftime("%Y-%m-%d")),
}

def get_meta(df, date_tuple, tournament_format):
    return df[
        (df['Format'] == tournament_format) 
        & (df['Date'] >= date_tuple[0])
        & (df['Date'] < date_tuple[1])
        ]

In [34]:
get_meta(df, date_tuple=releases['ARC'], tournament_format='CC')

,Country,Date,Decklist,Event,Format,Hero,Result,deck_link,event_link
0,TW,2020-10-17,Huang Chih Wei Warrior Deck 17.10.20,Taiwan National Champs 2020,CC,Dorinthea,1st,https://fabtcg.com/decklists/huang-chih-wei-wa...,https://fabtcg.com/organised-play/2020/taiwan-...
1,TW,2020-10-17,Wang Chung Chi Warrior Deck 17.10.20,Taiwan National Champs 2020,CC,Dorinthea,2nd,https://fabtcg.com/decklists/wang-chung-chi-wa...,https://fabtcg.com/organised-play/2020/taiwan-...
2,TW,2020-10-17,Fan Keng Hsu Ninja Deck 17.10.20,Taiwan National Champs 2020,CC,Katsu,3rd,https://fabtcg.com/decklists/fan-keng-hsu-ninj...,https://fabtcg.com/organised-play/2020/taiwan-...
3,TW,2020-10-17,Hsiu Te Li Warrior Deck 17.10.20,Taiwan National Champs 2020,CC,Dorinthea,4th,https://fabtcg.com/decklists/hsiu-te-li-warrio...,https://fabtcg.com/organised-play/2020/taiwan-...
4,TW,2020-10-17,Chen Wei Li Mechanologist Deck 17.10.20,Taiwan National Champs 2020,CC,Dash,5th,https://fabtcg.com/decklists/chen-wei-li-mecha...,https://fabtcg.com/organised-play/2020/taiwan-...
5,TW,2020-10-17,Mingxian Chen Warrior Deck 17.10.20,Taiwan National Champs 2020,CC,Dorinthea,6th,https://fabtcg.com/decklists/mingxian-chen-war...,https://fabtcg.com/organised-play/2020/taiwan-...
6,TW,2020-10-17,Chen Chih Hao Ninja Deck 17.10.20,Taiwan National Champs 2020,CC,Katsu,7th,https://fabtcg.com/decklists/chen-chih-hao-nin...,https://fabtcg.com/organised-play/2020/taiwan-...
7,TW,2020-10-17,Limbo_CFP Ninja Deck 17.10.20,Taiwan National Champs 2020,CC,Katsu,8th,https://fabtcg.com/decklists/limbo_cfp-ninja-d...,https://fabtcg.com/organised-play/2020/taiwan-...
8,MY,2020-10-11,Alvin Khaw Warrior Deck 11.10.20,Malaysia - Road to …,CC,Dorinthea,1st,https://fabtcg.com/decklists/alvin-khaw-warrio...,https://fabtcg.com/organised-play/2020/road-na...
9,MY,2020-10-11,Khoo Ne Bing Ninja Deck 11.10.20,Malaysia - Road to …,CC,Katsu,2nd,https://fabtcg.com/decklists/khoo-ne-bing-ninj...,https://fabtcg.com/organised-play/2020/road-na...


In [11]:
cru_df = get_meta(df, date=CRU_release, tournament_format="CC")

In [12]:
def add_decks_to_meta(meta_df):
    meta_list = meta_df.to_dict(orient='records')
    for i in meta_list:
        if i['deck_link'] is None:
            print("decklink is None")
        else:
            i['metadata'], i['deck'] = get_deck(i['deck_link'])
    
    return meta_list

In [13]:
def get_meta_share(df):
    hero_counts = df['Hero'].value_counts()
    hero_counts_df = pd.DataFrame({'hero': hero_counts.index, 'count': hero_counts, 'percentage': hero_counts / hero_counts.sum()})
    
    return hero_counts_df

In [14]:
cru_with_decks = add_decks_to_meta(cru_df)
print("done!")
len(cru_with_decks)

done!


44

In [15]:
cru_breakdown = get_meta_share(cru_df)
cru_breakdown

,hero,count,percentage
Katsu,Katsu,12,0.272727
Dorinthea,Dorinthea,11,0.250000
Dash,Dash,8,0.181818
Rhinar,Rhinar,8,0.181818
Bravo,Bravo,4,0.090909
Azalea,Azalea,1,0.022727


In [18]:
def count_card_group(group):
    
    # name = group.iloc[0]['name']
    total_copies = group['copies'].sum()
    if group[group['resource'] == 1].shape[0] > 0:
        red_copies = group[group['resource'] == 1].iloc[0]['copies']
    else:
        red_copies = 0
    if group[group['resource'] == 2].shape[0] > 0:
        yellow_copies = group[group['resource'] == 2].iloc[0]['copies']
    else:
        yellow_copies = 0
    if group[group['resource'] == 3].shape[0] > 0:
        blue_copies = group[group['resource'] == 3].iloc[0]['copies']
    else:
        blue_copies = 0

    return pd.DataFrame.from_dict({
        # 'name': name,
        'total_copies': [total_copies],
        'red_copies': [red_copies],
        'yellow_copies': [yellow_copies],
        'blue_copies': [blue_copies],
        })

def get_card_counts(deck, split_equipment=False):
    
    if split_equipment is False:
        counts = deck.groupby('name').apply(count_card_group)
        counts.reset_index(inplace=True)
        counts.drop('level_1', axis=1, inplace=True)

        return counts
    else:  # split_equipment is True
        pass

In [21]:
def get_staples(meta_list):
    counts = []
    for i in meta_list:
        counts.append(get_card_counts(i['deck']))
    c_concat = pd.concat(counts)
    v_counts = c_concat['name'].value_counts().rename_axis('name').reset_index(name='decks')
    staples = c_concat.groupby('name').mean()
    staples = staples.merge(v_counts, how='outer', on="name")
    staples.rename(columns={"decks_x": "decks"}, inplace=True)
    staples.sort_values('decks', ascending=False, inplace=True)
    staples['percentage_of_decks'] = staples['decks'] / len(meta_list)

    return staples


In [22]:
get_staples(cru_with_decks).to_csv("staples.csv")

In [23]:
staples = get_staples(cru_with_decks)

In [24]:
staples.head(10)

,name,total_copies,red_copies,yellow_copies,blue_copies,decks,percentage_of_decks
138,Sink Below,3.225000,2.950000,0.200000,0.075000,40,0.909091
27,Command and Conquer,2.818182,2.818182,0.000000,0.000000,33,0.750000
55,Fyendal's Spring Tunic,1.000000,0.000000,0.000000,0.000000,32,0.727273
119,Remembrance,1.357143,0.000000,1.357143,0.000000,28,0.636364
47,Enlightened Strike,2.964286,2.964286,0.000000,0.000000,28,0.636364
4,Arcanite Skullcap,1.000000,0.000000,0.000000,0.000000,27,0.613636
92,Nullrune Gloves,1.000000,0.000000,0.000000,0.000000,24,0.545455
136,Sigil of Solace,2.545455,2.545455,0.000000,0.000000,22,0.500000
170,Unmovable,4.000000,1.809524,0.904762,1.285714,21,0.477273
48,Fate Foreseen,3.450000,2.850000,0.450000,0.150000,20,0.454545
